In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
#from fbprophet import Prophet
import pickle
import copy
from tqdm.auto import tqdm

In [3]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: total: 750 ms
Wall time: 789 ms


In [4]:
''' fill na '''
for plc in plc_lst:
    df = df_dic[plc]
    df_dic[plc] = df_dic[plc].fillna(method='ffill')
    df_dic[plc] = df_dic[plc].fillna(method='bfill')
    df_dic[plc] = df_dic[plc].dropna()

In [5]:
%%time
''' create datetime '''
for plc in plc_lst:
    df_dic[plc]['date'] = df_dic[plc]['ds'].apply(lambda x : datetime.strptime(x , '%Y%m%d'))
    df_dic[plc]['year'] = pd.DatetimeIndex(df_dic[plc]['date']).year - 2010
    df_dic[plc]['month'] = pd.DatetimeIndex(df_dic[plc]['date']).month / 12
    df_dic[plc]['day'] = pd.DatetimeIndex(df_dic[plc]['date']).day / 31

CPU times: total: 6.03 s
Wall time: 6.04 s


---
# 데이터 만들기

In [35]:
def create_lag_feature(df_trg , lag_count , interval):
    df_n = df_trg.copy()
    for n in range(1, lag_count + 1):
        df_shift = []
        for col in fct_lst:
            df_shift.append( df_n[col].shift(n*interval).to_frame(name= f"{col}_lag{n*interval}") )
        df_n  = pd.concat( [df_n] + df_shift , axis = 1)
    dftemp = df_n.iloc[lag_count*interval:].dropna()
    #dftemp = df_n.iloc[n_lags*interval:-1]
    
    x = dftemp.iloc[:-1].drop(['ds','date'],axis = 1)
    y = dftemp[fct_lst].shift(-1).dropna()
    y.columns = [ 'y_' + x for x in y.columns ]
    return x, y

In [36]:
%%time
lag_count = 120
interval = 5

xs = dict()
ys = dict()
for plc in tqdm(plc_lst):
    x ,y = create_lag_feature(df_dic[plc] , lag_count , interval)
    xs[plc] = x
    ys[plc] = y


CPU times: total: 2min 12s
Wall time: 2min 13s


In [37]:
''' 저장 '''
pickle.dump([xs,ys] , open('./mydata/lstm_xy.plk','wb'))